In [3]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import requests
import time
from tqdm import tqdm
import os
import re
import datetime

import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException # 검색 결과가 없는 경우 예외 처리
from bs4 import BeautifulSoup

In [1]:
# 검색할 키워드 가져오기
# conn = db_connection('')
# news_crawl = pd.read_sql(f"select * from db_name", conn)
# conn.close()

# news_crawl.head()

# 키워드 예시: \콘서트\" -교회 -과학 -취업"

In [3]:
# 수집 날짜 지정(전일자)
before_one_day = datetime.datetime.now() - datetime.timedelta(days=1)
D, d = before_one_day.strftime("%Y.%m.%d"), before_one_day.strftime("%Y%m%d")
db_date = d[2:]

---

In [270]:
# 뉴스 링크 담을 DataFrame 생성
news_href = pd.DataFrame(columns = ['ck_idx', 'news', 'title', 'href', 'is_status'])

drv = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

for i in range(len(news_crawl)):
    crawl = True
    drv.get(f'https://search.naver.com/search.naver?where=news&query={news_crawl["search_word"][i]}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={D}&de={D}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{d}to{d}&is_sug_officeid=0&office_category=0&service_area=0')
    time.sleep(1)
    
    while crawl == True:
        soup = BeautifulSoup(drv.page_source, 'html.parser')
        try:
            list_news = soup.find('ul', {'class': 'list_news'})
            get_list = list_news.findAll('li', {'class':'bx'})

            # 해당 페이지 뉴스 기관, 기사 제목, 기사 링크 수집
            for lenth in tqdm(range(len(get_list))):
                title = get_list[lenth].find('a', {"class":"news_tit"})['title'].strip()
                
                # 네이버 뉴스 링크 없는 경우 존재
                try:
                    href = get_list[lenth].findAll('a', {'class':'info'})[-1]['href']
                    is_status = 1
                    if href.split('/')[2] != 'n.news.naver.com':
                        href = get_list[lenth].find('a', {'class':'news_tit'})['href']
                        is_status = 0
                except:
                    href = get_list[lenth].find('a', {'class':'news_tit'})['href']
                    is_status = 0
                
                # 기사 제목 접근 2가지 경우 존재
                try:
                    news_name = get_list[lenth].find('div', {'class':'info_group'}).find('a').get_text()
                except:
                    news_name = get_list[lenth].find('span', {'class':'info press'}).get_text()

                data = pd.DataFrame({'ck_idx':news_crawl['ck_idx'][i], 'news':[news_name], 'title':[title], 'href':[href], 'is_status':is_status})
                news_href = pd.concat([news_href, data], ignore_index=True)

            # 마지막 페이지인지 구분
            try:
                if len(drv.find_element(By.CLASS_NAME, 'api_sc_page_wrap').find_element(By.CLASS_NAME, 'btn_next').get_attribute('href')) > 0:
                    pass
            except:
                crawl = False
                break
        except:
            break
        
        # 페이지 이동
        # drv.get('https://m.search.naver.com/search.naver'+soup.find('a', {'class':'btn_next'})['href'])
        drv.find_element(By.CLASS_NAME, 'api_sc_page_wrap').find_element(By.CLASS_NAME, 'btn_next').click()
        time.sleep(3)
        
drv.close()

100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 800.13it/s]


100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 576.48it/s]


100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 815.14it/s]


100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 682.82it/s]


100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 736.92it/s]


100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 526.59it/s]


In [267]:
# 중복 제거
print('중복 제거 전 :', len(news_href))
news_href.drop_duplicates(['news', 'title'], ignore_index = True, inplace=True)
print('중복 제거 후 :', len(news_href))

In [ ]:
# 저장
news_href.to_xlsx(f'naver_news_{d}.xlsx')

In [271]:
# user_nm = db_id
# passwd = db_pw
# host_url = host
# port_num = num
# db_name = db_name
# engine = create_engine('')
# engine_conn = engine.connect()

# # insert
# news_href.to_sql(db_name, con=engine, if_exists='append', index=None)
# engine_conn.close()
# engine.dispose()